# 問題文
二次元グリッドの原点 
$(0, 0)$ にチェスのナイトの駒があります。

ナイトの駒はマス $(i,j)$ にあるとき $(i+1,j+2)$ か $(i+2,j+1)$ のどちらかのマスにのみ動かすことができます。

ナイトの駒をマス $(X,Y)$ まで移動させる方法は何通りありますか？

$10^9+7$ で割った余りを求めてください。

# 制約
$1 \leq X \leq 10^6$<br>
$1 \leq Y \leq 10^6$

入力中のすべての値は整数である。


# 最終的に通ったコード

In [1]:
import numpy as np


MOD = 10**9 + 7

X, Y = map(int, input().split())

if (X + Y) % 3 != 0:
    print(0)

else:
    if (Y < X/2) or (Y > 2*X):
        print(0)
    
    else:
      
        h = (2 * X - Y) // 3
        v = (2 * Y - X) // 3

        def extgcd(a, b):
            if b == 0:
                x = 1
                y = 0
            else:
                y, x = extgcd(b, a%b)
                y -= a//b * x
            return x, y

        def low_inverse(a):
            ret, _ = extgcd(a, MOD)
            return ret

        def low_combination(m, n):
            # calculate (m+n)!/n!m!
            ret = 1
            numerator = m + n
            for i in range(min(m, n)):
                inv_i = low_inverse(i + 1)
                ret = ret * (numerator - i) * inv_i % MOD
            return ret

        print(low_combination(h, v))


999999 999999
151840682


# 解法のポイント
- **到達不能な点を除外する**
    - knightの1回の移動量は常に3なので $X+Y$ が3の倍数でない場合は到達不可
    - 1回の移動で横移動および縦移動が少なくとも1発生するので $Y<\frac{1}{2}X$ あるいは $Y>2X$ の場合は到達不可
    - それ以外の場合はおそらく到達可能
- **到達可能な場合、$(X, Y)$ に到達する移動コマンドの数を求める**
    - $(2, 1)$ 移動を $h$、$(1, 2)$ 縦移動を $v$ とすると、これらが満たすべき条件は、$2h + v = X$ かつ $h + 2v = Y$ でこれは解ける
- **求めた $h$ と $v$ の組み合わせ数 $\frac{(h + v)!}{h!v!}$ を求める**
    - $(X, Y)$ が十分大きい時、これをナイーブに求めると発散するので $10^9 + 7$ で割りながら桁数を調整する
    - この時 $10^9+7$ の法の下での分母の逆数を求める必要がある

## Given Information

In [2]:
MOD = 10**9 + 7

X = 999999
Y = 999999

## 具体的解法案

### 到達不能な点を除外する
2つ目の条件は $h$ か $v$ が負になったら $0$ を返すでもたぶん同じ

In [3]:
if (X + Y) % 3 != 0:
    print(0)

In [4]:
if (Y < X/2) or (Y > 2*X):
    print(0)

### 到達可能な場合、$(X, Y)$ に到達する移動コマンドの数を求める

In [5]:
h = (2 * X - Y) // 3
v = (2 * Y - X) // 3

### 求めた $h$ と $v$ の組み合わせ数 $\frac{(h + v)!}{h!v!}$ を求める
組み合わせ数を高速に計算できる手法として、

- **二乗累乗法**
- **拡張ユークリッドの互除法**

が使えるっぽいので両方実装した。<br>
結論から言えば計算速度的に拡張ユークリッドの互除法一択。二進数分解の実装は改善できそうな気がせんでもない。

#### 二分累乗法による素数法の下での組み合わせ数計算の実装
$a^{p-2}$ が $p$ の法の下 $a$ の逆数になることを利用する。
- `MOD`は素数である必要がある
- `MOD`が巨大になってくると計算効率が悪そう

In [6]:
import numpy as np

# MOD<=1073741825 までなら対応可能 np.int64にすればもうちょいいける

def binary_expansion(x):
    power_2 = np.exp2(np.arange(31)).astype(np.int32)
    exp_list = [0] * np.argmin(x >= power_2)
    for i in reversed(range(len(exp_list))):
        if x >= power_2[i]:
            exp_list[i] = 1
            x -= power_2[i]
        else:
            continue
        if x == 0:
            break
    return exp_list

def low_inverse(a):
    e_list = binary_expansion(MOD-2)
    ret = 1
    for e in e_list:
        if e:
            ret = ret * a % MOD
        a = a * a % MOD
    return ret

def low_combination(m, n):
    # calculate (m+n)!/n!m!
    ret = 1
    numerator = m + n
    for i in range(min(m, n)):
        inv_i = low_inverse(i + 1)
        ret = ret * (numerator - i) * inv_i % MOD
    return ret

In [7]:
%%time
ret = low_combination(h, v)

CPU times: user 7.87 s, sys: 66.8 ms, total: 7.94 s
Wall time: 7.91 s


In [8]:
# 151840682
print(ret)

151840682


#### 拡張ユークリッドの互除法による組み合わせ数計算の実装
$ax + py = 1$ を満たす整数 $x$ を求める <br>
- 逆数計算以外は二分累乗法の実装と同じ

In [9]:
def extgcd(a, b):
    if b == 0:
        x = 1
        y = 0
    else:
        y, x = extgcd(b, a%b)
        y -= a//b * x
    return x, y

In [10]:
# (3, -11)
extgcd(111, 30)

(3, -11)

In [11]:
def low_inverse(a):
    ret, _ = extgcd(a, MOD)
    return ret

In [12]:
%%time
ret = low_combination(h, v)

CPU times: user 949 ms, sys: 3.11 ms, total: 952 ms
Wall time: 951 ms


In [13]:
# 151840682
ret

151840682

### 参考資料
- 「1000000007 で割ったあまり」の求め方を総特集！ 〜 逆元から離散対数まで 〜 <br>
https://qiita.com/drken/items/3b4fdf0a78e7a138cd9a
- 拡張ユークリッドの互除法 〜 一次不定方程式 ax + by = c の解き方 〜 <br>
https://qiita.com/drken/items/b97ff231e43bce50199a